# Data Acquisition Exercises

In [1]:
import os
import pandas as pd

import requests

1. Using the code from the lesson as a guide and the REST API from https://python.zgulde.net/api/v1/items as we did in the lesson, create a dataframe named items that has all of the data for items.

In [2]:
def get_api_df(domain, endpoint):
    url = domain + endpoint
    endpoint_split = endpoint.split('/')
    name = endpoint_split[-1]
    response = requests.get(url)
    data = response.json()
    return pd.DataFrame(data['payload'][name])

get_api_df('https://python.zgulde.net', '/api/v1/items')

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036
5,Sally Hansen,6,Sally Hansen Nail Color Magnetic 903 Silver El...,6.93,74170388732,74170388732
6,Twinings Of London,7,Twinings Of London Classics Lady Grey Tea - 20 Ct,9.64,70177154004,70177154004
7,Lea & Perrins,8,Lea & Perrins Marinade In-a-bag Cracked Pepper...,1.68,51600080015,51600080015
8,Van De Kamps,9,Van De Kamps Fillets Beer Battered - 10 Ct,1.79,19600923015,19600923015
9,Ahold,10,Ahold Cocoa Almonds,3.17,688267141676,688267141676


2. Do the same thing, but for stores (https://python.zgulde.net/api/v1/stores)

In [3]:
get_api_df('https://python.zgulde.net', '/api/v1/stores')

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218
5,1015 S WW White Rd,San Antonio,6,TX,78220
6,12018 Perrin Beitel Rd,San Antonio,7,TX,78217
7,15000 San Pedro Ave,San Antonio,8,TX,78232
8,735 SW Military Dr,San Antonio,9,TX,78221
9,8503 NW Military Hwy,San Antonio,10,TX,78231


3. Extract the data for sales (https://python.zgulde.net/api/v1/sales). There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

In [20]:
def get_api_df_with_next_page(domain, endpoint, print_prog = False):
    items = []
    endpoint_split = endpoint.split('/')
    name = endpoint_split[-1]
    i = 0
    while endpoint:
        url = domain + endpoint
        if print_prog:
            print(f"\rGetting info from {url}", end = '')
        response = requests.get(url)
        data = response.json()
        items.extend(data['payload'][name])
        # update the endpoint
        endpoint = data['payload']['next_page']
        i += 1
    return pd.DataFrame(items)

sales_df = get_api_df_with_next_page('https://python.zgulde.net', '/api/v1/sales', print_prog=True)

Getting info from https://python.zgulde.net/api/v1/sales?page=10

KeyboardInterrupt: 

In [26]:
sales_df

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218
5,1015 S WW White Rd,San Antonio,6,TX,78220
6,12018 Perrin Beitel Rd,San Antonio,7,TX,78217
7,15000 San Pedro Ave,San Antonio,8,TX,78232
8,735 SW Military Dr,San Antonio,9,TX,78221
9,8503 NW Military Hwy,San Antonio,10,TX,78231


4. Save the data in your files to local csv files so that it will be faster to access in the future.

In [24]:
def get_api_table_data(table_name, query_api=False, print_prog=False):
    '''Acquires the api data from the database or the .csv file if if is present

    Args:
        query_db = False (Bool) :  Forces a databse query and a resave of the data into a csv.
    Return:
        df (DataFrame) : a dataframe containing the data from the SQL database or the .csv file
    '''
    filename = f'{table_name}.csv'
    endpoints = {
        'items':'/api/v1/items',
        'stores':'/api/v1/stores',
        'sales':'/api/v1/sales'
    }
    #file name string literal
    #check if file exists and query_dg flag
    if os.path.isfile(filename) and not query_api:
        #return dataframe from file
        print(f'Returning saved csv file : {filename}')
        return pd.read_csv(filename).drop(columns = ['Unnamed: 0'])
    else:
        domain = 'https://python.zgulde.net'
        endpoint = endpoints[table_name]
        items = []
        i = 0
        while endpoint:
            url = domain + endpoint
            if print_prog:
                print(f'\rGetting info from {url}', end = '')
            response = requests.get(url)
            data = response.json()
            items.extend(data['payload'][table_name])
            # update the endpoint
            endpoint = data['payload']['next_page']
            i += 1
        df = pd.DataFrame(items)
        df.to_csv(filename)
        print(f'Saved as {filename}')
    return df

def get_all_tables(list_of_tables = ['items', 'stores', 'sales']):
    tables = dict()
    for t in list_of_tables:
        tables[t] = get_api_table_data(t)
        print(f"Got table: {t}")
    return tables

5. Combine the data from your three separate dataframes into one large dataframe.

In [7]:
def join_tables():
    tables_dict = get_all_tables()
    sales_df = tables_dict['sales']
    stores_df = tables_dict['stores']
    items_df = tables_dict['items']
    join_table = tables_dict['sales'].merge(tables_dict['items'], left_on='item', right_on='item_id')
    join_table = join_table.merge(tables_dict['stores'], left_on='store', right_on='store_id')
    return join_table
    
df = join_tables()

Returning saved csv file : items.csv
Got table: items
Returning saved csv file : stores.csv
Got table: stores
Returning saved csv file : sales.csv
Got table: sales


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 913000 entries, 0 to 912999
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   item           913000 non-null  int64  
 1   sale_amount    913000 non-null  float64
 2   sale_date      913000 non-null  object 
 3   sale_id        913000 non-null  int64  
 4   store          913000 non-null  int64  
 5   item_brand     913000 non-null  object 
 6   item_id        913000 non-null  int64  
 7   item_name      913000 non-null  object 
 8   item_price     913000 non-null  float64
 9   item_upc12     913000 non-null  int64  
 10  item_upc14     913000 non-null  int64  
 11  store_address  913000 non-null  object 
 12  store_city     913000 non-null  object 
 13  store_id       913000 non-null  int64  
 14  store_state    913000 non-null  object 
 15  store_zipcode  913000 non-null  int64  
dtypes: float64(2), int64(8), object(6)
memory usage: 118.4+ MB


6. Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

In [9]:
def get_german_energy_data(url='https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv'):
    df = pd.read_csv(url)
    return df

get_german_energy_data()

,Date,Consumption,Wind,Solar,Wind+Solar
0,2006-01-01,1069.18400,NaN,NaN,NaN
1,2006-01-02,1380.52100,NaN,NaN,NaN
2,2006-01-03,1442.53300,NaN,NaN,NaN
3,2006-01-04,1457.21700,NaN,NaN,NaN
4,2006-01-05,1477.13100,NaN,NaN,NaN
...,...,...,...,...,...
4378,2017-12-27,1263.94091,394.507,16.530,411.037
4379,2017-12-28,1299.86398,506.424,14.162,520.586
4380,2017-12-29,1295.08753,584.277,29.854,614.131
4381,2017-12-30,1215.44897,721.247,7.467,728.714


7. Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the acquire.py file and be able to re-run the functions and get the same data.

In [10]:
import acquire

In [11]:
acquire.join_tables(query_api_join=True).set_index('sale_id')

Saved as items.csv
Got table: items
Saved as stores.csv
Got table: stores
Saved as sales.csv
Got table: sales


,item,sale_amount,sale_date,store,item_brand,item_id,item_name,item_price,item_upc12,item_upc14,store_address,store_city,store_id,store_state,store_zipcode
sale_id,,,,,,,,,,,,,,,
1,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
2,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",1,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
3,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",1,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
4,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",1,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
5,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",1,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
912996,50,63.0,"Wed, 27 Dec 2017 00:00:00 GMT",10,Choice,50,Choice Organic Teas Black Tea Classic Black - ...,5.20,47445919221,47445919221,8503 NW Military Hwy,San Antonio,10,TX,78231
912997,50,59.0,"Thu, 28 Dec 2017 00:00:00 GMT",10,Choice,50,Choice Organic Teas Black Tea Classic Black - ...,5.20,47445919221,47445919221,8503 NW Military Hwy,San Antonio,10,TX,78231
912998,50,74.0,"Fri, 29 Dec 2017 00:00:00 GMT",10,Choice,50,Choice Organic Teas Black Tea Classic Black - ...,5.20,47445919221,47445919221,8503 NW Military Hwy,San Antonio,10,TX,78231
